In [47]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [2]:
load_dotenv()

True

In [8]:
API_BASE_URL = "https://api.twitterapi.io"
API_KEY = os.getenv("X_API_KEY")
HEADERS = {"X-API-Key": API_KEY}

def search_viral_tweets(query, min_likes=100, max_results=200, page_size=50):
    full_query = f"{query} min_faves:{min_likes} lang:en"
    url = f"{API_BASE_URL}/twitter/tweet/advanced_search"
    params = {
        "query": full_query,
        "max_results": page_size,
        "tweet.fields": "text,author_id,created_at,public_metrics,lang"
    }

    all_tweets = []
    next_cursor = ""

    while len(all_tweets) < max_results:
        if next_cursor:
            params["cursor"] = next_cursor

        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            raise Exception(f"Request failed: {response.status_code} {response.text}")

        data = response.json()
        tweets = data.get("tweets", [])
        all_tweets.extend(tweets)

        next_cursor = data.get("next_cursor") or ""
        if not next_cursor:
            break

    return all_tweets[:max_results]

def extract_viral_info(tweets):
    return [{
        "text": tweet["text"],
        "likes": tweet.get("public_metrics", {}).get("like_count", 0),
        "retweets": tweet.get("public_metrics", {}).get("retweet_count", 0),
        "replies": tweet.get("public_metrics", {}).get("reply_count", 0),
        "created_at": tweet.get("created_at"),
        "lang": tweet.get("lang")
    } for tweet in tweets]


query = '#Veo3 OR "Veo 3" OR "viral Veo3 ads" OR "#veo3prompt"'
viral_tweets = search_viral_tweets(query, min_likes=100, page_size=50)
viral_info = extract_viral_info(viral_tweets)

print("Top viral tweets:")
for idx, tweet in enumerate(viral_info, 1):
    print(f"{idx}. [{tweet['lang']}] {tweet['text']}")
    print(f"   Likes: {tweet['likes']}, Retweets: {tweet['retweets']}, Replies: {tweet['replies']}, Created at: {tweet['created_at']}\n")

Top viral tweets:
1. [en] TalkMe released the FIRST brand film made with Google's AI stack (Gemini, Veo 3 and Flow)

From a "Penguin" to a Million Learners: TalkMe Could Be the Next-Gen Duolingo
   Likes: 0, Retweets: 0, Replies: 0, Created at: None

2. [en] Veo 3 rate limits (free, pro, ultra) are doubling for the next 24 hours, go make something cool!!
   Likes: 0, Retweets: 0, Replies: 0, Created at: None

3. [en] Free users are getting +3 VEO 3 video generations extra during this weekend on Gemini! Pro and Ultra limits are doubled too! 

Veokend 👀👀👀 https://t.co/nbMiTeeUl9
   Likes: 0, Retweets: 0, Replies: 0, Created at: None

4. [en] New to making videos in Gemini? Here are some Veo 3 prompting tips to get you started 🧵
   Likes: 0, Retweets: 0, Replies: 0, Created at: None

5. [en] This weekend only, everyone gets 3 free #Veo3 video generations from Gemini. To help you make the most of it, we’ve pulled together a few tips from our team so you can get better outputs for your prom

In [10]:
df = pd.DataFrame(viral_info)
df.shape

(200, 6)

In [11]:
df.head()

,text,likes,retweets,replies,created_at,lang
0,TalkMe released the FIRST brand film made with...,0,0,0,None,en
1,"Veo 3 rate limits (free, pro, ultra) are doubl...",0,0,0,None,en
2,Free users are getting +3 VEO 3 video generati...,0,0,0,None,en
3,New to making videos in Gemini? Here are some ...,0,0,0,None,en
4,"This weekend only, everyone gets 3 free #Veo3 ...",0,0,0,None,en


In [130]:
scene_llm = ChatOpenAI(model="gpt-4.1", temperature=0.5)
script_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.7)
extraction_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

In [13]:
tweet_prompt_classifier = PromptTemplate(
    input_variables=["tweet_text"],
    template="""
You are an assistant that determines whether a tweet contains meaningful content that can be turned into a video generation prompt.

Instructions:
- If the tweet includes descriptive text, imagery, scene ideas, or visualizable content, classify as "YES".
- If the tweet is purely promotional, contains only links, hashtags, offers, or generic announcements without descriptive content, classify as "NO".
- Only return "YES" or "NO". Do not explain.

Example 1:
Tweet: "A serene forest with morning sunlight streaming through the trees. Perfect scene for a calming video."
Output: YES

Example 2:
Tweet: "This weekend only, everyone gets 3 free #Veo3 video generations from Gemini. Check it out ⬇️"
Output: NO

---

Tweet: {tweet_text}
Output:
"""
)


In [14]:
for idx, row in df.iterrows():
    classifier_chain = tweet_prompt_classifier | llm | StrOutputParser()
    result = classifier_chain.invoke({"tweet_text": row['text']})
    df.at[idx, 'is_meaningful'] = result

In [159]:
df_meaningful = df[df['is_meaningful'] == 'YES']

In [173]:
df.to_csv('../data/viral_tweets_classified.csv', index=False)

In [20]:
df_meaningful.head()

,text,likes,retweets,replies,created_at,lang,is_meaningful
7,"Veo-3 is freaking amazing, can't imagine what ...",0,0,0,None,en,YES
10,🎨 VEO 3 1080 P🎨\n\n Prompt :\n\nTracking sho...,0,0,0,None,en,YES
17,Veo -3 fast on Gemini 🛶\n\nA serene cinematic ...,0,0,0,None,en,YES
20,"🎨 VEO 3 🎨\n\nJSON Prompt :\n\n{\n ""shot"": {...",0,0,0,None,en,YES
21,"Veo-3 fast on Gemini 🏕️\n\n""A hyper-realistic,...",0,0,0,None,en,YES


In [140]:
df_meaningful.shape

(70, 8)

In [46]:
from pydantic import BaseModel, Field
from typing import List

class Scene(BaseModel):
    index: int = Field(..., description="Scene number from 1 to 8")
    description: str = Field(..., description="Compelling, cinematic, viral-potential scene description")

class Storyboard(BaseModel):
    scenes: List[Scene] = Field(..., description="List of 8 sequential scenes that form a viral short story")


In [ ]:
output_parser = JsonOutputParser(pydantic_object=Storyboard)

In [ ]:
scene_generation_prompt = PromptTemplate(
    input_variables=["tweet"],
    template="""
    # Enhanced Viral Ad Director Prompt

You are an elite viral advertising director with expertise in creating emotionally compelling, shareable commercial content that drives massive engagement and conversions. Your specialty is transforming social media insights into cinematic advertisements that go viral while seamlessly integrating brand messaging and driving action.

## YOUR MISSION
Transform the provided tweet data into a captivating, emotionally charged 1-minute viral advertisement that maximizes shareability, brand awareness, and conversion potential through masterful storytelling, visual composition, and strategic product integration.

## ADVERTISEMENT SPECIFICATIONS
- **Duration**: Exactly 60 seconds (8 scenes × 7.5 seconds each)
- **Structure**: 8 sequential scenes that build narrative momentum toward purchase/action
- **Tone**: Cinematic quality with viral-worthy emotional hooks and subtle brand integration
- **Target**: Multi-platform optimization (TikTok, Instagram Reels, YouTube Shorts, Facebook)
- **Goal**: Drive viral sharing AND measurable business outcomes

## VIRAL ADVERTISING FRAMEWORK

### Narrative Arc for Conversions
- **Scenes 1-2**: Hook & Problem Identification (capture attention, establish relatable pain point)
- **Scenes 3-4**: Emotional Amplification (deepen connection, build desire for solution)
- **Scenes 5-6**: Solution Reveal (introduce product/service as hero, demonstrate value)
- **Scenes 7-8**: Transformation & Call-to-Action (show results, compel immediate action)

### Commercial Integration Strategy
- **Organic Product Placement**: Naturally weave product into storyline
- **Emotional Association**: Connect brand with positive feelings and outcomes
- **Social Proof**: Include authentic user reactions and testimonials
- **Scarcity/Urgency**: Create compelling reasons to act now
- **Clear Value Proposition**: Communicate unique benefits within narrative

## SCENE CONSTRUCTION REQUIREMENTS

### Advanced Commercial Storytelling
Each scene description must include:

1. **Camera Work**: Specific shot types optimized for mobile viewing
2. **Lighting**: Mood that enhances product appeal and emotional connection
3. **Setting**: Environment that reinforces brand values and target audience
4. **Character Actions**: Authentic interactions that build trust and desire
5. **Product Integration**: Natural, non-intrusive brand/product presence
6. **Audio Strategy**: Music, sound effects, and dialogue that reinforce message
7. **Emotional Beat**: Specific feeling that drives toward conversion
8. **Transition**: Seamless flow that maintains engagement momentum

### Viral Advertising Triggers (Include 3-4 per video)
- **Problem-Solution Relief**: Pain point resolution that feels revolutionary
- **Social Validation**: "Everyone's doing this" moments that create FOMO
- **Transformation Reveals**: Before/after moments that inspire action
- **Insider Knowledge**: "Secret" tips that make viewers feel special
- **Authentic Testimonials**: Real people expressing genuine satisfaction
- **Lifestyle Aspiration**: Showing the life customers want to live
- **Humor with Purpose**: Comedy that enhances rather than distracts from message
- **Educational Value**: Teaching something useful while promoting product

## CONVERSION-FOCUSED ELEMENTS

### Psychology of Viral Ads
- **Pattern Interrupt**: Unexpected opening that stops scroll behavior
- **Story-Brand Fusion**: Seamlessly blend narrative with commercial message
- **Emotional Investment**: Make viewers care about outcome before revealing solution
- **Social Currency**: Create moments viewers want to share to look smart/helpful
- **Clear Benefit Communication**: Obvious value proposition within entertainment

### Strategic Brand Integration
- **Natural Product Placement**: Product appears as logical story element
- **Lifestyle Integration**: Show product enhancing desirable lifestyle
- **Problem-Solution Fit**: Product perfectly solves established problem
- **Trust Building**: Authentic moments that build brand credibility
- **Action Triggers**: Clear next steps for interested viewers

## OUTPUT FORMAT REQUIREMENTS

**CRITICAL**: Your response must be a valid Python dictionary with this exact structure:

```python
{{
  "scenes": [
    {{"index": 1, "description": "[Detailed scene 1 with hook, setting, camera work, character actions, brand integration approach, emotional beat, and transition cue - 150-250 words]"}},
    {{"index": 2, "description": "[Detailed scene 2 with problem amplification, product introduction, emotional development - 150-250 words]"}},
    {{"index": 3, "description": "[Detailed scene 3 continuing narrative with deeper product integration - 150-250 words]"}},
    {{"index": 4, "description": "[Detailed scene 4 with solution demonstration and value proof - 150-250 words]"}},
    {{"index": 5, "description": "[Detailed scene 5 with transformation beginning and social proof - 150-250 words]"}},
    {{"index": 6, "description": "[Detailed scene 6 with clear benefits and lifestyle enhancement - 150-250 words]"}},
    {{"index": 7, "description": "[Detailed scene 7 with compelling results and urgency building - 150-250 words]"}},
    {{"index": 8, "description": "[Detailed scene 8 with powerful call-to-action, final emotional punch, and share-worthy ending that drives conversions - 150-250 words]"}}
  ]
}}
```

### Format Rules (Non-Negotiable):
- Use double quotes for all strings and keys
- No markdown code blocks or backticks
- No explanatory text before or after the dictionary
- Each description must be 150-250 words
- Include specific brand integration in each scene
- End with clear call-to-action

## VIRAL AD SUCCESS METRICS

### Engagement Optimization
- **Stop Rate**: Irresistible opening that halts scrolling
- **Watch-Through Rate**: Each scene builds curiosity for the next
- **Emotional Resonance**: Moments that create strong feelings about brand
- **Share Impulse**: Content so compelling viewers must share immediately
- **Action Intent**: Clear desire to learn more or purchase

### Conversion Elements
- **Trust Signals**: Authentic moments that build brand credibility
- **Value Clarity**: Obvious benefits that justify action
- **Urgency Creation**: Compelling reasons to act now vs. later
- **Friction Reduction**: Clear, simple next steps
- **Social Proof**: Evidence others love this product/service

## BRAND INTEGRATION BEST PRACTICES
- **Subtlety Over Pushiness**: Natural placement that enhances story
- **Value-First Approach**: Help before selling
- **Emotional Connection**: Associate brand with positive feelings
- **Authentic Usage**: Show realistic product interaction
- **Clear Differentiation**: Highlight unique advantages
- **Memorable Messaging**: Key points that stick after viewing

## TWEET DATA TO TRANSFORM INTO VIRAL AD
{tweet}

Remember: Your goal is to create an advertisement so engaging and valuable that viewers share it willingly while developing strong positive associations with the brand and taking the desired action. Every scene must serve both entertainment and conversion objectives.
    """
)

In [55]:
tweet_to_multiscene_prompt = PromptTemplate(
    input_variables=["tweet_text", "scene_number", "scene_description", "previous_context"],
    template="""
You are a professional meta-prompt generator for Veo-3, leveraging the standardized, 7-component meta-prompt architecture 
to convert social media posts into cinematic, viral, multi-scene video ads.

**Mission**: Expand the given tweet into a **continuation scene** ({scene_number}) that follows seamlessly 
from the previous one, while still adhering to the viral-ad tone.

**Previous Context (for continuity):**  
"{previous_context}"

**Current Scene to Develop:**  
"{scene_description}"

---

### Core Rules
- Use the **7-Component Structure** precisely for this scene:
  1. **Subject** – Rich description (15+ attributes: age, gender, ethnicity, hair, eyes, build, clothing, posture, mannerisms, distinctive features, emotional state, voice style, etc.).
  2. **Action** – What the subject(s) are doing: gestures, movements, pacing, emotional changes.
  3. **Scene** – Environment details: location, props, lighting, time of day, atmosphere, textures.
  4. **Style** – Cinematic elements: camera placement _(use Veo-3 explicit camera syntax like: (close-up shot), (tracking shot), (aerial view))_, lighting style, color palette, visual tone.
  5. **Dialogue (optional)** – Use colon syntax, include tone indicators. (Character: "…").
  6. **Sounds** – Ambient audio, music, sound effects, voices, silence.
  7. **Technical (Negative Prompt)** – Explicitly list what to avoid: watermarks, text overlays, poor quality, distortions, artifacts, broken limbs, off-screen cuts, etc.

- Ensure:
  - Scene duration fits **≈8 seconds** but feels cinematic.  
  - Tone is **viral-ad style**: catchy, emotionally engaging, with a share-worthy quality.  
  - End with a **scene-level micro-hook** (mini call-to-action or emotional beat) that leads naturally into the next scene.  

---

### Output Format
Return strictly in Markdown:

```markdown
### Viral Veo-3 Video Ad Scene {scene_number} (Duration: ~8 s)

**Subject:**  
[Detailed subject description]

**Action:**  
[...]

**Scene:**  
[...]

**Style:**  
[...]

**Dialogue (if any):**  
[...]

**Sounds:**  
[...]

**Technical (Negative Prompt):**  
[...]

**Scene-Level Hook:**  
[A mini hook that ensures the viewer wants to see what happens next]
"""
)

In [59]:
scene_outputs

['### Viral Veo-3 Video Ad Scene 1 (Duration: ~8 s)\n\n**Subject:**  \nA teenage girl, approximately 17 years old, of Asian descent, with long, slightly messy black hair, expressive brown eyes filled with a mix of boredom and hope, slender build, casual hoodie and jeans, slouched posture, subtle freckles across her nose, wearing earbuds, soft features, voice gentle but slightly restless, exuding a quiet yearning for connection.\n\n**Action:**  \nShe slowly lowers her phone, her gaze drifting upward as if lost in thought, then subtly squeezes her eyes shut for a moment, deep in contemplation. Her fingers absentmindedly tap or twirl a loose strand of hair. Her shoulders slump slightly, conveying a sense of longing and frustration.\n\n**Scene:**  \nSet in a small, cozy bedroom at dusk; warm, low lighting casts gentle shadows. The background features a cluttered desk with scattered notebooks, a lamp, and a window showing a fading sunset. Textured walls with posters and string lights add de

In [119]:
import ast
import re

def normalize_scene_list(text):
    """
    Ensure LLM output is parsed as:
    [{'1': "..."}, {'2': "..."}, ..., {'8': "..."}]
    """
    if isinstance(text, list):
        return text

    if isinstance(text, str):
        # remove markdown fences if present
        text = re.sub(r"^```(?:python|json)?|```$", "", text.strip(), flags=re.MULTILINE).strip()
        try:
            parsed = ast.literal_eval(text)  # safer than eval, handles Python-style dicts
            if isinstance(parsed, list):
                return parsed
        except Exception as e:
            raise ValueError(f"Could not parse scene list:\n{text}") from e

    raise TypeError(f"Unexpected format for scenes_json: {type(text)}")

In [137]:
# prompt to extract JSON from the text
json_extraction_prompt = """
You are a JSON extraction expert. Your task is to extract a JSON object from the following text.

Text: {text}

Please respond with the extracted JSON object only, without any additional commentary or explanation.
format_instruction:
{format_instruction}
"""

json_extraction_prompt_tempelate = PromptTemplate(
    input_variables=["text"],
    template=json_extraction_prompt,
    partial_variables={"format_instruction": output_parser.get_format_instructions()}
)



In [138]:
def generate_video_script(tweet):

    scene_chain = scene_generation_prompt | scene_llm | StrOutputParser()
    json_chain = json_extraction_prompt_tempelate | extraction_llm | output_parser

    scenes_json = scene_chain.invoke({"tweet": tweet})
    scenes_json = json_chain.invoke({"text": scenes_json})

    print(scenes_json)
    
    scene_outputs = []
    previous_context = "Start of video"

    script_chain = tweet_to_multiscene_prompt | script_llm | StrOutputParser()

    for scene in scenes_json["scenes"]:
        scene_number = scene["index"]
        scene_description = scene["description"]

        result = script_chain.invoke({
            "scene_number": scene_number,
            "scene_description": scene_description,
            "previous_context": previous_context
        })

        scene_outputs.append(result)

        # Update context for continuity
        previous_context += f" Summary of Scene {scene_number}: {scene_description}..."
    
    return scene_outputs

In [163]:
df_meaningful = df_meaningful.sample(2)
for idx, row in df_meaningful.iterrows():
   result = generate_video_script(tweet=row['text'])
   df_meaningful.at[idx, 'video_script'] = result

{'scenes': [{'index': 1, 'description': 'Scene opens with a striking vertical close-up of golden sunlight breaking over endless desert dunes, dust motes swirling in slow motion. The camera, in ultra-high 8K, pans left to right, revealing a lone traveler silhouetted against the rising sun. The lighting is ethereal—warm, cinematic, and immersive—casting dramatic shadows that evoke a sense of awe and possibility. The setting is vast and atmospheric, inspired by Denis Villeneuve’s epic style, instantly transporting viewers to an otherworldly, aspirational landscape. Subtle, orchestral music swells as the traveler’s boots crunch softly on the sand. No product is shown yet, but a faint glint of metallic blue peeks from the traveler’s backpack, hinting at something extraordinary. The emotional beat is one of curiosity and anticipation, designed to interrupt scrolling and make viewers wonder: Who is this traveler? Where are they going? The transition is a seamless push-in as the traveler pause

In [164]:
df_meaningful.head()

,text,likes,retweets,replies,created_at,lang,is_meaningful,video_script
173,Veo 3 fast is so good\n\nPrompt:\n\nCinematic ...,0,0,0,None,en,YES,[### Viral Veo-3 Video Ad Scene 1 (Duration: ~...
112,Veo 3 can mix paint to make a new color\n\nsou...,0,0,0,None,en,YES,[### Viral Veo-3 Video Ad Scene 1 (Duration: ~...


In [165]:
df_meaningful['video_script'].iloc[0]

['### Viral Veo-3 Video Ad Scene 1 (Duration: ~8 s)\n\n**Subject:**  \nA young woman in her late 20s, ethnically diverse with a warm caramel complexion, long flowing chestnut hair catching the sunlight, piercing hazel eyes full of determination, athletic build, dressed in lightweight, breathable desert gear—tan cargo pants, a fitted white shirt, and a rugged leather jacket. She stands with a relaxed yet purposeful posture, shoulders squared, exuding confidence and curiosity. Her face shows a subtle smile, hinting at inner strength and adventure. A faint scar runs along her cheek, adding a layer of resilience.\n\n**Action:**  \nShe slowly raises her hand to shield her eyes from the sun, her gaze fixed on the distant horizon. As she pauses, her chest rises and falls steadily, embodying calm anticipation. Her fingers gently brush a metallic blue pendant hanging from her neck—subtle but significant—suggesting a personal connection or purpose. She takes a deliberate step forward, then stops

In [166]:
from google import genai
from google.genai import types
import time
from IPython.display import Video, display

PROJECT_ID = "test-prateek-bandi-5ocnz4"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")



def initialize_client():
    # Initialize the Gemini client
    client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
    return client


# video_model = "veo-3.0-generate-001"
# video_model_fast = "veo-3.0-fast-generate-001"
# image_video_model = "veo-3.0-generate-preview"
# gemini_model = "gemini-2.5-flash"

# def generate_video(prompt):
#     client = initialize_client()
#     operation = client.models.generate_videos(
#         model="veo-3.0-generate-001",
#         prompt=prompt,
#             config=types.GenerateVideosConfig(
#             aspect_ratio="16:9",
#             number_of_videos=1,
#             duration_seconds=20,
#             resolution="1080p",
#             person_generation="allow_adult",
#             enhance_prompt=True,
#             generate_audio=True,
#         ))

#     # Wait for the operation to complete
#     while not operation.done:
#         time.sleep(20)
#         operation = client.operations.get(operation)
#     return operation

# def save_video(operation, name: str):
#     if operation.response:
#         video_data = operation.response.generated_videos[0].video.video_bytes
#         # Save video to file
#         with open(f"../videos/{name}.mp4", "wb") as f:
#             f.write(video_data)
#         print(f"Video saved as {name}.mp4")

In [167]:
import cv2

In [168]:
# ========== Client Setup ==========
def initialize_client():
    return genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

client = initialize_client()

# Models
VIDEO_MODEL = "veo-3.0-generate-001"
VIDEO_MODEL_FAST = "veo-3.0-fast-generate-001"
IMAGE_VIDEO_MODEL = "veo-3.0-generate-preview"
GEMINI_MODEL = "gemini-2.5-flash"


# ========== Generation Functions ==========
def generate_video(prompt, duration=8, resolution="1080p", starting_image=None):
    """Generate a single video from a text prompt (optionally seeded with an image)."""
    operation = client.models.generate_videos(
        model=VIDEO_MODEL,
        prompt=prompt,
        image=types.Image.from_file(location=starting_image) if starting_image else None,
        config=types.GenerateVideosConfig(
            aspect_ratio="16:9",
            number_of_videos=1,
            duration_seconds=duration,
            resolution=resolution,
            person_generation="allow_adult",
            enhance_prompt=True,
            generate_audio=True,
        )
    )

    # Wait for completion
    while not operation.done:
        time.sleep(10)
        operation = client.operations.get(operation)

    return operation


def save_video(operation, name: str, out_dir):
    """Save the completed video operation to file."""
    os.makedirs(out_dir, exist_ok=True)

    if operation.response:
        video_data = operation.response.generated_videos[0].video.video_bytes
        video_path = os.path.join(out_dir, f"{name}.mp4")

        with open(video_path, "wb") as f:
            f.write(video_data)

        print(f"✅ Video saved: {video_path}")
        return video_path
    else:
        print(f"⚠️ No response for {name}")
        return None


def extract_last_frame(video_path, out_dir):
    """Extract the last frame of a video and save as an image."""
    if video_path is None:
        return None

    os.makedirs(out_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"❌ Could not open video: {video_path}")
        return None

    # Jump to last frame
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

    ret, frame = cap.read()
    cap.release()

    if ret:
        frame_path = os.path.join(
            out_dir,
            os.path.basename(video_path).replace(".mp4", "_lastframe.jpg")
        )
        cv2.imwrite(frame_path, frame)
        print(f"🖼️ Last frame saved: {frame_path}")
        return frame_path
    else:
        print(f"⚠️ Failed to read last frame: {video_path}")
        return None


# ========== Run for DataFrame ==========
def process_dataframe(df, out_video_root="../videos", out_frame_root="../frames", resume=True):
    video_paths = []
    frame_paths = []

    for idx, scenes in df["video_script"].items():
        if not isinstance(scenes, list):
            print(f"Skipping row {idx}, invalid scenes format")
            video_paths.append(None)
            frame_paths.append(None)
            continue

        # Create per-row directories
        row_video_dir = os.path.join(out_video_root, str(idx))
        row_frame_dir = os.path.join(out_frame_root, str(idx))
        os.makedirs(row_video_dir, exist_ok=True)
        os.makedirs(row_frame_dir, exist_ok=True)

        row_videos, row_frames = [], []
        starting_image = None

        for s_idx, scene in enumerate(scenes, start=1):
            video_name = f"scene{s_idx}"
            video_path = os.path.join(row_video_dir, f"{video_name}.mp4")
            frame_path = os.path.join(row_frame_dir, f"{video_name}_lastframe.jpg")

            if resume and os.path.exists(video_path) and os.path.exists(frame_path):
                print(f"⏩ Skipping {video_path}, already exists")
                row_videos.append(video_path)
                row_frames.append(frame_path)
                starting_image = frame_path
                continue

            # Generate video
            operation = generate_video(scene, starting_image=starting_image)

            # Save video
            video_path = save_video(operation, video_name, row_video_dir)
            row_videos.append(video_path)

            # Save last frame
            frame_path = extract_last_frame(video_path, row_frame_dir)
            row_frames.append(frame_path)

            # Pass last frame as starting image for next scene
            starting_image = frame_path

        video_paths.append(row_videos)
        frame_paths.append(row_frames)

    df["video_paths"] = video_paths
    df["last_frame_paths"] = frame_paths
    return df

In [169]:
df_meaningful = process_dataframe(df_meaningful)

✅ Video saved: ../videos/173/scene1.mp4
🖼️ Last frame saved: ../frames/173/scene1_lastframe.jpg
✅ Video saved: ../videos/173/scene2.mp4
🖼️ Last frame saved: ../frames/173/scene2_lastframe.jpg
✅ Video saved: ../videos/173/scene3.mp4
🖼️ Last frame saved: ../frames/173/scene3_lastframe.jpg
✅ Video saved: ../videos/173/scene4.mp4
🖼️ Last frame saved: ../frames/173/scene4_lastframe.jpg
✅ Video saved: ../videos/173/scene5.mp4
🖼️ Last frame saved: ../frames/173/scene5_lastframe.jpg
✅ Video saved: ../videos/173/scene6.mp4
🖼️ Last frame saved: ../frames/173/scene6_lastframe.jpg
✅ Video saved: ../videos/173/scene7.mp4
🖼️ Last frame saved: ../frames/173/scene7_lastframe.jpg
✅ Video saved: ../videos/173/scene8.mp4
🖼️ Last frame saved: ../frames/173/scene8_lastframe.jpg
✅ Video saved: ../videos/112/scene1.mp4
🖼️ Last frame saved: ../frames/112/scene1_lastframe.jpg


[h264 @ 0x131165910] mmco: unref short failure
[h264 @ 0x131165910] mmco: unref short failure


✅ Video saved: ../videos/112/scene2.mp4
🖼️ Last frame saved: ../frames/112/scene2_lastframe.jpg


[h264 @ 0x1307d2d50] mmco: unref short failure


✅ Video saved: ../videos/112/scene3.mp4
🖼️ Last frame saved: ../frames/112/scene3_lastframe.jpg
✅ Video saved: ../videos/112/scene4.mp4
🖼️ Last frame saved: ../frames/112/scene4_lastframe.jpg
✅ Video saved: ../videos/112/scene5.mp4
🖼️ Last frame saved: ../frames/112/scene5_lastframe.jpg
✅ Video saved: ../videos/112/scene6.mp4
🖼️ Last frame saved: ../frames/112/scene6_lastframe.jpg
✅ Video saved: ../videos/112/scene7.mp4
🖼️ Last frame saved: ../frames/112/scene7_lastframe.jpg
✅ Video saved: ../videos/112/scene8.mp4
🖼️ Last frame saved: ../frames/112/scene8_lastframe.jpg


In [171]:
from moviepy import VideoFileClip, concatenate_videoclips
import os

def stitch_videos(video_list, output_path):
    """Stitch a list of videos into one continuous video."""
    clips = []
    for v in video_list:
        if v and os.path.exists(v):
            clips.append(VideoFileClip(v))
        else:
            print(f"⚠️ Skipping missing video: {v}")
    if not clips:
        print(f"❌ No valid videos to stitch for {output_path}")
        return None

    final_clip = concatenate_videoclips(clips, method="compose")
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    print(f"🎬 Stitched video saved: {output_path}")
    return output_path


def stitch_dataframe_videos(df, out_dir="../stitched_videos"):
    """Stitch all scene videos row-wise into one video per tweet row."""
    os.makedirs(out_dir, exist_ok=True)
    stitched_paths = []

    for idx, row in df.iterrows():
        video_list = row.get("video_paths", [])
        tweet_dir = os.path.join(out_dir, f"tweet{idx}")
        os.makedirs(tweet_dir, exist_ok=True)

        stitched_path = os.path.join(tweet_dir, f"tweet{idx}_final.mp4")
        stitched = stitch_videos(video_list, stitched_path)
        stitched_paths.append(stitched)

    df["stitched_video"] = stitched_paths
    return df

In [172]:
df_meaningful = process_dataframe(df_meaningful)
df_meaningful = stitch_dataframe_videos(df_meaningful)

⏩ Skipping ../videos/173/scene1.mp4, already exists
⏩ Skipping ../videos/173/scene2.mp4, already exists
⏩ Skipping ../videos/173/scene3.mp4, already exists
⏩ Skipping ../videos/173/scene4.mp4, already exists
⏩ Skipping ../videos/173/scene5.mp4, already exists
⏩ Skipping ../videos/173/scene6.mp4, already exists
⏩ Skipping ../videos/173/scene7.mp4, already exists
⏩ Skipping ../videos/173/scene8.mp4, already exists
⏩ Skipping ../videos/112/scene1.mp4, already exists
⏩ Skipping ../videos/112/scene2.mp4, already exists
⏩ Skipping ../videos/112/scene3.mp4, already exists
⏩ Skipping ../videos/112/scene4.mp4, already exists
⏩ Skipping ../videos/112/scene5.mp4, already exists
⏩ Skipping ../videos/112/scene6.mp4, already exists
⏩ Skipping ../videos/112/scene7.mp4, already exists
⏩ Skipping ../videos/112/scene8.mp4, already exists
MoviePy - Building video ../stitched_videos/tweet173/tweet173_final.mp4.
MoviePy - Writing audio in tweet173_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ../stitched_videos/tweet173/tweet173_final.mp4



MoviePy - Done !
MoviePy - video ready ../stitched_videos/tweet173/tweet173_final.mp4
🎬 Stitched video saved: ../stitched_videos/tweet173/tweet173_final.mp4
MoviePy - Building video ../stitched_videos/tweet112/tweet112_final.mp4.
MoviePy - Writing audio in tweet112_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ../stitched_videos/tweet112/tweet112_final.mp4



MoviePy - Done !
MoviePy - video ready ../stitched_videos/tweet112/tweet112_final.mp4
🎬 Stitched video saved: ../stitched_videos/tweet112/tweet112_final.mp4
